In [1]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache"
os.environ["AUTH_TOKEN"] = "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"
import datasets
dataset_dict = datasets.load_dataset("hotpot_qa", "distractor")

Using the latest cached version of the module from /home/olab/tomerronen1/xdg_cache/huggingface/modules/datasets_modules/datasets/hotpot_qa/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5 (last modified on Thu Sep 22 14:56:17 2022) since it couldn't be found locally at /home/olab/tomerronen1/git_repos/last_projects_playground/long_rangeness/hotpot_qa/hotpot_qa.py, or remotely (FileNotFoundError).
Reusing dataset hotpot_qa (/home/olab/tomerronen1/xdg_cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
example = dataset_dict["train"][6]
print(example["question"])
print(example["answer"])
print(example["supporting_facts"])
for title, sentence_id in zip(example["supporting_facts"]["title"], example["supporting_facts"]["sent_id"]):
    paragraph_id = example["context"]["title"].index(title)
    print(example["context"]["sentences"][paragraph_id][sentence_id])

Which genus of moth in the world's seventh-largest country contains only one species?
Crambidae
{'title': ['Indogrammodes', 'Indogrammodes', 'India', 'India'], 'sent_id': [0, 1, 0, 1]}
Indogrammodes is a genus of moths of the Crambidae family.
 It contains only one species, Indogrammodes pectinicornalis, which is found in India.
India, officially the Republic of India ("Bhārat Gaṇarājya"), is a country in South Asia.
 It is the seventh-largest country by area, the second-most populous country (with over 1.2 billion people), and the most populous democracy in the world.


In [3]:
import sled
from transformers import AutoModelForSeq2SeqLM, AutoConfig
model = AutoModelForSeq2SeqLM.from_pretrained("tau/bart-base-sled",
                                              use_auth_token="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")


Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [5]:
model

SledForConditionalGeneration(
  (_underlying_model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50265, 768, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
        (layers): ModuleList(
          (0): BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, 